<a href="https://colab.research.google.com/github/JIWON0520/TIL/blob/main/Alexnet_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
!mkdir /content/drive/MyDrive/Alexnet

mkdir: cannot create directory ‘/content/drive/MyDrive/Alexnet’: File exists


In [3]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
#image 256X256사이즈로 변경후 227X227로 crop
transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.RandomCrop(227),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset=datasets.CIFAR10('/content/drive/MyDrive/Alexnet',download=True,train=True,transform=transform)
trainset=DataLoader(trainset,batch_size=128,shuffle=True)
testset=datasets.CIFAR10('/content/drive/MyDrive/Alexnet',download=True,train=False,transform=transform)
testset=DataLoader(testset,batch_size=128,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
from torch.nn.modules.pooling import MaxPool2d
class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet,self).__init__()

    self.convLayer=nn.Sequential(
        #Conv 1: Conv,ReLU,LRN,pool
        nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,padding=0,stride=4),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2),

        #Conv 2: Conv,ReLU,LRN,pool
        nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,padding=2,stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2),

        #Conv 3: Conv,ReLU
        nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,padding=1,stride=1),
        nn.ReLU(inplace=True),

        #Conv 4: ConV,ReLU
        nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,padding=1,stride=1),
        nn.ReLU(inplace=True),

        #Conv 5: Conv,ReLU,LRN,pool
        nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,padding=1,stride=1),
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2)
    )

    self.fcLayer=nn.Sequential(
        #fc 1: fc,ReLU,dropout
        nn.Linear(in_features=(256*6*6),out_features=4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5,inplace=True),

        #fc 2:fc,ReLU,dropout
        nn.Linear(in_features=4096,out_features=4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5,inplace=True),

        #fc 3:fc
        nn.Linear(in_features=4096,out_features=10)
    )

    for layer in self.convLayer:
      if isinstance(layer,nn.Conv2d):
        nn.init.normal_(layer.weight,mean=0,std=0.01)
        nn.init.constant_(layer.bias,0)

    nn.init.constant_(self.convLayer[4].bias,1)
    nn.init.constant_(self.convLayer[10].bias,1)
    nn.init.constant_(self.convLayer[12].bias,1)

  def forward(self,train):
    output=self.convLayer(train)
    output=output.view(-1,256*6*6)
    output=self.fcLayer(output)
      
    return output

In [24]:
alexnet=AlexNet().to(device)

#Optimizer 생성
#learning rate=0.01, momentum=0.9, weigth decay=0.0005
optimizer=optim.SGD(alexnet.parameters(),lr=0.01,momentum=0.9,weight_decay=0.0005)
criterion=nn.CrossEntropyLoss().to(device)

In [20]:
epochs=5
iteration=len(trainset)
for epoch in range(epochs):
  epoch_loss=0
  for data,classes in trainset:
    inputs,labels=data.to(device),classes.to(device)

    optimizer.zero_grad()
    outputs=alexnet(inputs)

    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    epoch_loss+=loss.item()
    print(loss.item())
  print('epoch {0} loss:{1:.3f}'.format(epoch,epoch_loss/iteration))


2.3287136554718018
2.3345582485198975
2.345329761505127
2.3476603031158447
2.319033145904541
2.3137118816375732
2.332839250564575
2.314603567123413
2.3244614601135254
2.3237144947052
2.304042339324951
2.3226699829101562
2.3042683601379395
2.2942044734954834
2.3294451236724854
2.2821857929229736
2.3092265129089355
2.296130418777466
2.36281156539917
2.3219425678253174
2.329435110092163
2.3191287517547607
2.318396806716919
2.3211677074432373
2.2908082008361816
2.3477766513824463
2.3166439533233643
2.3109068870544434
2.328735113143921
2.321251392364502
2.3158555030822754
2.320390224456787


KeyboardInterrupt: ignored

In [13]:
alexnet.eval()

with torch.no_grad():
  total=0.0
  accuracy=0.0
  for data,classes in testset:
    inputs,labels=data.to(device),classes.to(device)

    outputs=alexnet(inputs)

    _,pred=torch.max(outputs.data,1)
    total+=outputs.size(0)
    accuracy+=(pred==labels).sum().item()
print((accuracy/total)*100)

10.0
